In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pickle
pickle_file_2= open('/content/drive/MyDrive/pickle1_baseline-2', "rb")       
tokenized_corpus = pickle.load(pickle_file_2)
pickle_file_2.close()


In [3]:
import numpy as np
!pip install rank_bm25
from rank_bm25 import BM25Okapi
import pickle

def flatten(x):
    result = []
    for el in x:
        if hasattr(el, "__iter__") and not isinstance(el, str):
            result.extend(flatten(el))
        else:
            result.append(el)
    return result



tokenized_corpus=flatten(tokenized_corpus)
bm25 = BM25Okapi(tokenized_corpus)
print(len(tokenized_corpus))

def search(search_tokens, num_results=10):
    scores = bm25.get_scores(search_tokens)
    top_indexes = np.argsort(scores)[::-1][:num_results]
    
    return top_indexes,scores

tokenized_query='according to china'
top_indexes,scores=search(tokenized_query)
print(bm25.get_top_n(tokenized_query, tokenized_corpus, n=1))
print(top_indexes)

1556162
['hochgradig']
[1341900  198503  126694  126532 1067328 1160458  126653 1034859  477840
  589072]


**Baseline2_Bert_Model**

In [4]:
!pip install sentence_transformers
import numpy as np
import pandas as pd
import scipy as sc

import os
import json
import warnings

from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

warnings.filterwarnings("ignore")






     |████████████████████████████████| 81kB 5.2MB/s 
     |████████████████████████████████| 2.0MB 8.3MB/s 
     |████████████████████████████████| 1.2MB 42.5MB/s 
     |████████████████████████████████| 890kB 39.7MB/s 
     |████████████████████████████████| 3.2MB 44.7MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-1.0.3-cp37-none-any.whl size=114277 sha256=557ced5e78909a4af9fd4c0c74168520423948ab36a0bab2d3e419feaf311317
  Stored in directory: /root/.cache/pip/wheels/c8/0d/fa/4e28cf045da4781344e7972befb2fdf306051b225bfc290187
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=b76bc3e0844ee1e1ee7b9d0e08b28d3a35be75767309c8b4805231419bff9df2
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sentence-transformers sacremoses


In [5]:
pickle_file_1 = open('/content/drive/MyDrive/pickle2_baseline-2', "rb")       
model = pickle.load(pickle_file_1)
abstract_embeddings = pickle.load(pickle_file_1)
df = pickle.load(pickle_file_1)
pickle_file_1.close()


In [6]:
question_embedding = model.encode(['What do we know about virus genetics, origin, and evolution?'])

queries = ["Does AHCCCS have a centralized resource for medical coding resources related to COVID-19?",
               "Are there billing codes available for COVID-19 testing outside of Centers for Disease Control and Prevention (CDC) testing laboratories?",
"Will AHCCCS issue guidance regarding prior authorization expectations related to COVID-19 testing and treatment?",
"Is there a claims modifier for services related to the COVID-19 emergency?",
"Does AHCCCS cover testing for COVID-19?",
"Is there an ICD-10 diagnostic code for COVID-19?",
"How should hospitals that are reimbursed by APR-DRG bill for inpatient services related to the new ICD-10 diagnosis code for COVID-19?",
"Is IHS required to report CPT and modifiers on UB-04 claims (pertaining to COVID-19)?",
"Does AHCCCS reimburse IHS and 638 providers for services rendered at an Alternate Care Site (ACS)?",
"Can providers bill Medicaid for personal protective equipment (PPE)?",
"Does AHCCCS have guidance about Prior Authorization (PA) and Concurrent Review procedures during the COVID-19 emergency?",
"Is AHCCCS willing to suspend medical record review audits and behavioral health chart audit review processes at this time?",
"Will AHCCCS consider relaxing the timeframe for issuing Notice of Adverse Benefit Determination (NOABD) letters for non-urgent prior authorization requests?",
"Due to the need to print securely we have been unable to deploy these staff home while complying with this AHCCCS requirement?"
"Has AHCCCS received any additional grant funding to help providers address COVID-19 emergency?",
"Where can I get information and updates about the coronavirus and COVID-19?",
"Where can providers find further information about the CARES and Families First Coronavirus Response Acts?",
"Is COVID-19 laboratory testing available outside of the ADHS State Lab?",
"Is AHCCCS covering COVID-19 antibody testing?",
"Who does ADHS notify when there is a confirmed positive case of COVID-19?",
"Does AHCCCS have a centralized resource for members who have general questions or are experiencing flu-like symptoms?",
"Who is at high risk for serious illness?",
"Should I keep doctor appointments for routine visits?",
"What is the guidance on well-child visits during the COVID-19 pandemic?",
"Should elective and non-emergency medical procedures be delayed or postponed?",
"Arizonans are understandably stressed and anxious right now. What mental health resources do you recommend?",
"What businesses are deemed â€œessentialâ€ under the Governorâ€™s Executive Order issued March 23, 2020?",
"Will members lose coverage during the COVID-19 emergency?",
"Are non-Medicaid members who are covered under state-only funding, also protected from disenrollment?",
"Is AHCCCS continuing to process Medicaid renewals during the COVID-19 emergency?",
"Should providers assist members with their renewals?",
"Will AHCCCS reinstitute Prior Quarter coverage during the declared emergency?",
"Is AHCCCS aware of any opportunities for providers to secure loans during the COVID emergency?",
"During the COVID-19 emergency will AHCCCS members be required to pay premiums?",
"How does AHCCCS plan to use the additional 6.2 % in federal matching funds appropriated for Medicaid programs under the Families First Coronavirus Response Act?",
"Does the one-time COVID-19 stimulus payment and/or the unemployment benefit increase affect AHCCCS eligibility?",
"Due to the COVID-19 emergency, CMS is granting State Medicaid programs the flexibility to waive and/or modify certain Medicaid requirements. What flexibilities has AHCCCS requested?",
"How does Governor Duceyâ€™s March 24th Executive Order freezing all evictions in Arizona related to COVID for 120 days affect persons determined Seriously Mentally Ill (SMI) or persons with behavioral health needs (GMH/SU) in AHCCCS supported residential programs, including Permanent Supportive Housing, and/or Behavioral Health Residential Facilities (BHRFs), in which members pay room and board?",
"Will cell phone text and minute limits be increased or lifted for the free or low cost cell phones provided by the federal government under the Lifeline Act?",
"Medicaid applicants need to sign the AHCCCS application form. How is a signature captured when an individual applies online or by phone and does not submit a paper application?",
"Should Health Plans be educating their members and contracted providers about COVID-19?",
"Should Health Plans and FFS providers continue routine staff visits to healthcare facilities, provider practices, and membersâ€™ homes?",
"Should Health Plans continue to go onsite to provider locations to investigate immediate quality of care concerns?",
"Should ALTCS Health Plans and Tribal ALTCS programs continue to conduct in-person case management and support coordination visits with members?",
"Are Health Plans permitted to conduct targeted outreach to members at risk to ensure they know the warning signs, understand precautions, and are prepared to take appropriate action should they become ill?",
"Is the newborn notification requirement continuing during the COVID19 emergency?",
"Will AHCCCS release rules that allow SNF-based dialysis instead of only free standing dialysis centers?",
"Will AHCCCS consider waiving or suspending certain deliverables to allow greater flexibility to MCOs considering much of our workforce is telecommuting and focused on member care?",
"Do the Health Plans need to request and review COOP documents from providers?",
"Is AHCCCS willing to halt hybrid performance measure efforts in light of COVID-19?",
"Are there any changes to timeframes for behavioral health service delivery during the COVID-19 emergency for members in DCS custody?",
"Will AHCCCS consider relaxing the mandatory 72-hour turnaround time for Behavioral Health Residential Facility (BHRF) prior authorization (PA) determinations due to high volume and possible capacity issues?",
"Will Corporate Compliance Audits be suspended during the COVID-19 emergency?"]

query_embeddings = model.encode(queries)

In [7]:
question_abstract = []
bert_scores = []

for abstract_embedding, abstract_text in zip(abstract_embeddings, df['abstract']):
    bert_score = cosine_similarity([question_embedding[0], abstract_embedding])[1][0]
    question_abstract.append(bert_score)

In [17]:
list1 = []
closest_n = 1
for query, query_embedding in zip(queries, query_embeddings):
    distances = sc.spatial.distance.cdist([query_embedding], abstract_embeddings, "cosine")[0]

    results = zip(range(len(distances)), distances)
    results = sorted(results, key=lambda x: x[1])

    print("\n\n======================\n\n")
    print("Query:", query)
    print("\nTop 5 most similar sentences in corpus:")

    for idx, distance in results[0:closest_n]:
        print(df['abstract'][idx].strip(), "\n(Score: %.4f)" % (1-distance),"\n")
        list1.append(df['abstract'][idx].strip())





Query: Does AHCCCS have a centralized resource for medical coding resources related to COVID-19?

Top 5 most similar sentences in corpus:
signs symptoms determine patient presenting primary care hospital outpatient settings covid 19 disease 
(Score: 0.7539) 





Query: Are there billing codes available for COVID-19 testing outside of Centers for Disease Control and Prevention (CDC) testing laboratories?

Top 5 most similar sentences in corpus:
clinical scoring system yet established estimate likelihood coronavirus disease determine suitability diagnostic testing suspected covid 19 methods single center observational suspected covid 19 confirmed covid 19 patient clinical course laboratory computed tomography ct findings presence alternative diagnoses evaluated clinical risk scores developed based clinical differences without covid 19 
(Score: 0.7673) 





Query: Will AHCCCS issue guidance regarding prior authorization expectations related to COVID-19 testing and treatment?

Top 5 

In [18]:
list1

['signs symptoms determine patient presenting primary care hospital outpatient settings covid 19 disease',
 'clinical scoring system yet established estimate likelihood coronavirus disease determine suitability diagnostic testing suspected covid 19 methods single center observational suspected covid 19 confirmed covid 19 patient clinical course laboratory computed tomography ct findings presence alternative diagnoses evaluated clinical risk scores developed based clinical differences without covid 19',
 'given predicted need continued sars cov 2 diagnostic testing evolving availability types diagnostic tests site covid 19 testing centers osctc leaders need timely guidance ensure meeting needs unique populations research discusses challenges offers considerations healthcare organizations others setting running osctcs also provides springboard engage policy makers leaders healthcare community discussion emergency preparedness better respond testing needs going forward',
 'aims document i

In [19]:
list_question = ["Does AHCCCS have a centralized resource for medical coding resources related to COVID-19?",
               "Are there billing codes available for COVID-19 testing outside of Centers for Disease Control and Prevention (CDC) testing laboratories?",
"Will AHCCCS issue guidance regarding prior authorization expectations related to COVID-19 testing and treatment?",
"Is there a claims modifier for services related to the COVID-19 emergency?",
"Does AHCCCS cover testing for COVID-19?",
"Is there an ICD-10 diagnostic code for COVID-19?",
"How should hospitals that are reimbursed by APR-DRG bill for inpatient services related to the new ICD-10 diagnosis code for COVID-19?",
"Is IHS required to report CPT and modifiers on UB-04 claims (pertaining to COVID-19)?",
"Does AHCCCS reimburse IHS and 638 providers for services rendered at an Alternate Care Site (ACS)?",
"Can providers bill Medicaid for personal protective equipment (PPE)?",
"Does AHCCCS have guidance about Prior Authorization (PA) and Concurrent Review procedures during the COVID-19 emergency?",
"Is AHCCCS willing to suspend medical record review audits and behavioral health chart audit review processes at this time?",
"Will AHCCCS consider relaxing the timeframe for issuing Notice of Adverse Benefit Determination (NOABD) letters for non-urgent prior authorization requests?",
"Due to the need to print securely we have been unable to deploy these staff home while complying with this AHCCCS requirement?"
"Has AHCCCS received any additional grant funding to help providers address COVID-19 emergency?",
"Where can I get information and updates about the coronavirus and COVID-19?",
"Where can providers find further information about the CARES and Families First Coronavirus Response Acts?",
"Is COVID-19 laboratory testing available outside of the ADHS State Lab?",
"Is AHCCCS covering COVID-19 antibody testing?",
"Who does ADHS notify when there is a confirmed positive case of COVID-19?",
"Does AHCCCS have a centralized resource for members who have general questions or are experiencing flu-like symptoms?",
"Who is at high risk for serious illness?",
"Should I keep doctor appointments for routine visits?",
"What is the guidance on well-child visits during the COVID-19 pandemic?",
"Should elective and non-emergency medical procedures be delayed or postponed?",
"Arizonans are understandably stressed and anxious right now. What mental health resources do you recommend?",
"What businesses are deemed â€œessentialâ€ under the Governorâ€™s Executive Order issued March 23, 2020?",
"Will members lose coverage during the COVID-19 emergency?",
"Are non-Medicaid members who are covered under state-only funding, also protected from disenrollment?",
"Is AHCCCS continuing to process Medicaid renewals during the COVID-19 emergency?",
"Should providers assist members with their renewals?",
"Will AHCCCS reinstitute Prior Quarter coverage during the declared emergency?",
"Is AHCCCS aware of any opportunities for providers to secure loans during the COVID emergency?",
"During the COVID-19 emergency will AHCCCS members be required to pay premiums?",
"How does AHCCCS plan to use the additional 6.2 % in federal matching funds appropriated for Medicaid programs under the Families First Coronavirus Response Act?",
"Does the one-time COVID-19 stimulus payment and/or the unemployment benefit increase affect AHCCCS eligibility?",
"Due to the COVID-19 emergency, CMS is granting State Medicaid programs the flexibility to waive and/or modify certain Medicaid requirements. What flexibilities has AHCCCS requested?",
"How does Governor Duceyâ€™s March 24th Executive Order freezing all evictions in Arizona related to COVID for 120 days affect persons determined Seriously Mentally Ill (SMI) or persons with behavioral health needs (GMH/SU) in AHCCCS supported residential programs, including Permanent Supportive Housing, and/or Behavioral Health Residential Facilities (BHRFs), in which members pay room and board?",
"Will cell phone text and minute limits be increased or lifted for the free or low cost cell phones provided by the federal government under the Lifeline Act?",
"Medicaid applicants need to sign the AHCCCS application form. How is a signature captured when an individual applies online or by phone and does not submit a paper application?",
"Should Health Plans be educating their members and contracted providers about COVID-19?",
"Should Health Plans and FFS providers continue routine staff visits to healthcare facilities, provider practices, and membersâ€™ homes?",
"Should Health Plans continue to go onsite to provider locations to investigate immediate quality of care concerns?",
"Should ALTCS Health Plans and Tribal ALTCS programs continue to conduct in-person case management and support coordination visits with members?",
"Are Health Plans permitted to conduct targeted outreach to members at risk to ensure they know the warning signs, understand precautions, and are prepared to take appropriate action should they become ill?",
"Is the newborn notification requirement continuing during the COVID19 emergency?",
"Will AHCCCS release rules that allow SNF-based dialysis instead of only free standing dialysis centers?",
"Will AHCCCS consider waiving or suspending certain deliverables to allow greater flexibility to MCOs considering much of our workforce is telecommuting and focused on member care?",
"Do the Health Plans need to request and review COOP documents from providers?",
"Is AHCCCS willing to halt hybrid performance measure efforts in light of COVID-19?",
"Are there any changes to timeframes for behavioral health service delivery during the COVID-19 emergency for members in DCS custody?",
"Will AHCCCS consider relaxing the mandatory 72-hour turnaround time for Behavioral Health Residential Facility (BHRF) prior authorization (PA) determinations due to high volume and possible capacity issues?",
"Will Corporate Compliance Audits be suspended during the COVID-19 emergency?"]

In [20]:
list_question=[el.replace('\xa0',' ') for el in list_question]

In [21]:
answer_list1 = list1

In [22]:
mannual_answer=["Yes, the AHCCCS Medical Coding Resources webpage includes a COVID-19 Medical Coding Information section and COVID-19 Emergency Medical Coding guidance.",
"Yes. The Centers for Medicare & Medicaid Services (CMS) created a new Healthcare Common Procedure Coding System (HCPCS) code for COVID-19 tests administered outside of Centers for Disease Control and Prevention (CDC) testing laboratories. This code corresponds with a Food and Drug Administration policy that allows certain laboratories to develop their own tests. Providers should use existing HCPCS code U0001 for CDC testing laboratories and the new HCPCS code U0002 for non-CDC lab tests. In addition, the American Medical Association has added a new Current Procedural Terminology (CPT) code: 87635 infectious agent detection by nucleic acid (DNA or RNA); severe acute respiratory syndrome coronavirus 2 (SARS-CoV-2) (Coronavirus disease [COVID-19]), amplified probe technique."
"Prior authorization is not permitted for COVID-19 testing or treatment.",
"Yes, AHCCCS has designated the CR modifier to be used on all claims for services provided as a result of, or related to COVID-19. It is imperative that providers begin utilizing this modifier immediately in all appropriate instances in order for AHCCCS to identify the costs of services attributable to this emergency. All other guidance regarding use of modifiers continues to be applicable.",
"Yes, AHCCCS covers COVID-19 testing. HCPCS U0001 and U0002 have an effective date of February 4, 2020. CPT 87635 has an effective date of March 13, 2020. HCPCS U0003 and U0004 have effective dates of March 18, 2020. All codes have been entered in the AHCCCS PMMIS system.",
"Yes, The World Health Organization has developed an emergency ICD-10 code for the coronavirus: U07.1, 2019-nCoV acute respiratory disease. The U07.1 is effective in PMMIS for dates of service February 4, 2020 and thereafter.  Additionally, CDCâ€™s National Center for Health Statistics will implement a new diagnosis code into the ICD 10th Revision, Clinical Modification, effective with the next update on October 1, 2020; CDC issued interim coding guidance and guidelines for health care encounters and deaths related to COVID-19.",
"AHCCCS has been notified that the 3M software will not recognize the new ICD-10 diagnosis code for COVID-19 prior to April 1, 2020.  Until the April 1, 2020 software release, hospitals should bill for other related conditions such as: Pneumonia,Acute Bronchitis,Lower Respiratory Infection,ARDS - Acute respiratory distress syndrome.The April 1, 2020 release of 3M software will recognize the newly defined ICD-10 diagnosis code, U07.1, mapped to an existing APR-DRG code under the April 1, 2020 software release.",
"HS facilities are not required to report CPT/HCPCS and modifiers on UB-04 claim forms when billing the All Inclusive Rate (AIR).",
"Per the COVID-19 Emergency Declaration Blanket Waivers & Flexibilities for Health Care Providers document released by CMS, AHCCCS will reimburse for services provided at or through an ACS, with a retroactive effective date of March 1, 2020 through the end of the emergency declaration.  Such services shall be reimbursable, so  long as those services performed are administered to a Title XIX or Title XXI eligible AHCCCS member, and are medically necessary, cost-effective, federally and state reimbursable, provided by an AHCCCS-registered provider.",
"There are no AHCCCS billing codes to cover the cost of purchasing PPE. However, some health care plans are offering enhanced provider payments and supplying PPE to providers in need. Some provider associations are also able to offer PPE. If you are in need of PPE see FAQ #4 under Provider Enrollment & Requirements.",
"Yes, please see the AHCCCS May 1, 2020 memo. This guidance does not apply to AHCCCS Fee for Service Programs. Fee for Service guidance, in coordination with tribal stakeholders, is available in this Fee for Service memo.",
"AHCCCS temporarily suspended medical record review/behavioral health chart audit efforts for providers without quality/utilization concerns through the end of the COVID-19 emergency.  While this will continue for the vast majority of audits, AHCCCS has reinstated the System of Care Practice Review (SOCPR) process effective June 1, 2020.",
"No, AHCCCS will not be extending the timeframe for Notice of Adverse Benefit Determination (NOABD) letters for non-urgent prior authorization requests.",
"Yes, the Substance Abuse and Mental Health Services Administration (SAMHSA) awarded a $2 million grant to AHCCCS on April 16, 2020 to increase mental health services infrastructure in response to the COVID-19 national emergency. Funding will be distributed through the Tribal Regional Behavioral Health Authorities/Regional Behavioral Health Authorities  to serve individuals with co-occurring Serious Mental Illness (SMI) designations and Substance Use Disorder (SUD) service support needs. The grant requires: 70 percent to provide direct services to those with co-occurring SMI and SUD needs; 10 percent for healthcare workers with mental health needs (less severe than SMI) requiring mental health care as a result of COVID-19; and 20 percent for all other individuals with mental health needs less severe than SMI.",
"Information about COVID-19 is available from the Centers for Disease Control (CDC) and the Arizona Department of Health Services.  Additionally, the general public in Arizona can call the Arizona Poison Control System at 1-844-542-8201 or the statewide COVID-19 Hotline, available 8 a.m. to 8 p.m. daily, in English and Spanish, by dialling 2-1-1 in Arizona. The COVID-19 hotline is administered by 2-1-1 Arizona and the Crisis Response Network, and can address questions about how to prepare for and prevent COVID-19 spread; testing information for COVID-19; at-risk populations; what to do if an individual gets sick; COVID-19 and animals; and other resources for accurate, reliable, and up-to-date information.",
"CARES and FFCR Act guidance and FAQs on Medicaid.gov.",
"Yes, as of March 16, 2020 COVID-19 testing is now available through private labs as well as the state lab. However, the COVID-19 test requires a specimen to be collected by a physician or other authorized healthcare providers. Do not visit a laboratory location to request the COVID-19 test. Instead, contact your physician or other authorized healthcare provider for guidance regarding getting tested. Your healthcare provider can best advise whether you meet criteria for testing and their office's availability of testing supplies.",
"Yes, as outlined in Section 6004 of the Families First Coronavirus Response Act (FFCRA). Further information is available at the CDC and on the ADHS websites.",
"ADHS notifies the memberâ€™s ordering provider and the memberâ€™s County Public Health Department.",
"Yes. AHCCCS has created a web page to address Medicaid-related questions from providers and contractors about COVID-19 at azahcccs.gov/AHCCCS/AboutUs/covid19. ",
"According to the CDC's Situation Summary,, pandemic COVID-19 poses a serious public health risk. Per the Arizona Department of Health Services Frequently Asked.",
"Please call your health care provider regarding scheduled, routine visits.",
"Please see the most recent guidelines from the American Academy of Pediatrics.",
"Following a CMS recommendation, Governor Doug Ducey announced an Executive Order on March 19, 2020 that halts all elective surgeries in the state of Arizona in order to free up medical resources and maintain the capacity for hospitals and providers to continue offering vital services. On April 22, the Governor announced an Executive Order removing restrictions on elective surgeries. Effective May 1, hospitals, health care facilities and providers that meet certain preparedness criteria can resume elective surgeries. Arizona Department of Health Services has posted FAQs about elective surgeries and  an exemption request form.",
"AHCCCS covers a full array of behavioral health services for members, and crisis services are available to anyone in Arizona, regardless of insurance coverage. In addition, we have added behavioral health resources offered by the CDC and SAMHSA on the COVID-19 webpage.",
"To stem the spread of COVID-19, Governor Ducey issued an executive order clarifying businesses and operations that are deemed â€œessentialâ€ by the state, including, but are not limited to, the following roles and/or job functions: Healthcare and Public Health Operations: Hospitals, clinics, dental offices, pharmacies.",
"No, per the Families First Coronavirus Response Act, AHCCCS will not disenroll members during the emergency (with the exception of death, those who move out of state, or those who voluntarily withdraw).",
"No, members who are not federally funded under Title XIX (for example, members with a Serious Mental Illness determination who do not qualify for Medicaid) are not exempt from disenrollment during the public health emergency.",
"Yes, while we are not disenrolling members (with the exception of death, those who move out of state, or those who voluntarily withdraw), we are continuing to process annual renewals. Providers are encouraged to continue to assist members with their renewals.",
"Prior Quarter, or retroactive coverage, is currently available to children under age 19, pregnant women and women in a post partum period.  For individuals other than children, pregnant women and women in a post partum period, approved applications are effective  as of the first day of the month in which the application is submitted.  Individuals are encouraged to submit an application immediately if they are experiencing a health issue, become uninsured or experience a change in circumstance that may make them eligible for AHCCCS Medical Assistance. Apply for AHCCCS.",
"On March 27, 2020, the Coronavirus Aid, Relief, and Economic Security Act of 2020 (CARES) passed, including immediate loan assistance for small businesses. Providers may wish to review loan opportunities authorized under the Act to determine if they qualify for this stimulus funding. Stimulus loans are not under the purview of the Medicaid program and AHCCCS does not play a role in the administration of small business loan assistance.",
"No.  For the duration of the COVID-19 emergency, AHCCCS has suspended premium payments for the KidsCare and Freedom to Work programs as well as copayments for the Transitional Medical Assistance (TMA) and Adult populations.  Members who have already paid March 2020 premiums will be credited, and individuals who were disenrolled in March will be reinstated for the duration of the emergency.",
"We are aware of stakeholder interest in how AHCCCS plans to use the enhanced federal funding available through the Families First Coronavirus Response Act.  We continue to evaluate how to best use the funds, in light of the expected member growth due to the COVID-19 emergency.  In order to receive the match, AHCCCS is required not to disenroll members, and expects membership growth as the existing members remain enrolled and new members are added.  As such, AHCCCS must first ensure that the additional funding will cover the cost of the enrollment growth that will occur. AHCCCS is prohibited from disenrolling members through the end of the month in which the declaration ends.",
"The one-time COVID-19 stimulus payments and the Unemployment Insurance Federal Pandemic Unemployment Compensation (FPUC) benefits will not affect AHCCCS eligibility. See the Arizona Department of Economic Service PEUC website for more information on how FPUC, Unemployment Insurance, and Pandemic Unemployment Assistance payments may affect Medicaid eligibility.",
"AHCCCS has requested various flexibilities to waive and/or modify certain Medicaid requirements under the 1135 and 1115 Waivers and the State Plan Amendment. CMS has begun to review and grant approval of these requests. As approval is granted, AHCCCS is making operational decisions regarding implementation, and tracking progress in the Status of AHCCCS Emergency Authority Requests document on this page.",
"AHCCCS intends for the Executive Order to apply to all AHCCCS and/or Medicaid funded permanent supportive and transitional housing programs as well as services settings in which AHCCCS members pay for room and board such as BHRFs, assisted living facilities, and group homes. Residential stability is critical for members to comply with mandated stay at home orders. To this end, there should be no disruption of service or alteration of existing subsidies for persons residing in AHCCCS funded Permanent Supportive and Transitional Living programs for persons determined SMI. Similarly, these protections should be broadly applied to any service setting whereby AHCCCS members pay for room and board.",
"In all situations during this crisis, AHCCCS encourages increased care coordination of members with behavioral health needs in housing or residential programs/settings to ensure housing/placement security during this crisis.  This will allow members to maintain their residential setting and comply with social distancing and stay at home practices required under our State and National COVID response orders for their own health and the health of our community.",
"AHCCCS has learned that some Lifeline Act cell phone providers are lifting limits and providing unlimited minutes and texting due to the COVID-19 emergency. Information is available at: www.obamaphone.com and www.usac.org/lifeline.",
"When an individual applies using the HEAplus.gov online application, the system captures an electronic signature. When an individual applies over the phone, the state employee or community assistor captures a voice signature. During the COVID-19 public health emergency, community assistors are authorized to obtain verbal consent to act on the applicant's behalf and complete the application.",
"Yes, Health Plans and FFS Programs should be proactively educating members and providers utilizing information from the Centers for Disease Control (CDC), the Arizona Department of Health Services (ADHS), and other applicable entities outlined in the AHCCCS Contractor Operations Manual (ACOM) 404, Attachment A.  If the only source of information in member education materials is one of the entities listed in  ACOM 404, Attachment A, it does not need to be submitted to AHCCCS for review or approval.",
"We highly recommend that, when feasible, both Health Plans and FFS Programs and providers leverage technological platforms to conduct virtual visits whenever possible.  For example, a 180 day case manager visit for an Arizona Long Term Care System (ALTCS) member located at a Skilled Nursing Facility (SNF) can be done via telehealth.",
"In accordance with CMS guidance, AHCCCS is requesting that Health Plans and FFS Programs maintain investigative efforts for serious health and safety threats (e.g. allegations of abuse). Health Plans and FFS Programs should do their part to support sound infection control protocols, including limiting the number of staff that go onsite for the investigation, implementation of preventive practices such as handwashing and use of hand sanitizer before engaging with members and staff, and assessment of staff health before entering the facility.   If Health  Plans and FFS Programs encounter issues with ability to conduct onsite/in-person investigation of immediate quality of care concerns, please notify AHCCCS QM of the limitation at CQM@azahcccs.gov.",
"Per AMPM 1620-E, case managers (and support coordinators) must conduct in-person visits for members residing in a skilled nursing facility every 180 days and to a member receiving HCBS or acute-only services every 90 days.  AHCCCS is suspending the in-person requirement at this time.  Case managers should continue to reach out to members telephonically to complete monitoring/reassessment efforts to the extent possible.  If members are not able to be reached via telephone or other electronic means, outreach attempts should be documented in the memberâ€™s case management file.",
"Yes, AHCCCS encourages health plans to conduct targeted outreach to at risk members.  We highly recommend that Health Plans leverage technological platforms to conduct virtual visits whenever possible.",
"The Newborn notification requirements have NOT changed and remain in place.",
"AHCCCS is adopting CMS guidance for dialysis treatment for the duration of the COVID emergency.  CMS is waiving the requirement that dialysis facilities have to provide services directly on its main premises or on other premises that are contiguous with the main premises. Dialysis facilities may enter nursing home/skilled nursing home facilities to provide dialysis service to its patients in those settings in order to limit community exposure. CMS continues to require that services provided to these nursing home residents are under the direction of the same governing body and professional staff as the residentâ€™s usual Medicare-certified dialysis facility. Further, in order to ensure that care is safe, effective and is provided by trained and qualified personnel, CMS requires that the dialysis facility staff: furnish all dialysis care and services, provide all equipment and supplies necessary, maintain equipment and supplies in the nursing home, and complete all equipment maintenance, cleaning and disinfection using appropriate infection control procedures and manufacturerâ€™s instructions for use. Dialysis services delivered in a nursing home or skilled nursing facility should continue to be billed to Medicare as appropriate for dual eligible members.",
"Yes, AHCCCS is currently evaluating where flexibility can be offered for deliverables.  Additional guidance will be issued in the near future.  If an MCO has a specific request, please send it to your AHCCCS Operations/Compliance Officer.",
"During the MCO-AHCCCS Weekly COVID-19 call on 3/16, Director Snyder conveyed the message that AHCCCS expects the health plans to check in with critical provider types to ensure these providers, such as Mobile Crisis providers, Stabilization Crisis providers, Long Term Care facilities, and Outpatient Treatment Program (OTP) providers have Continuity of Operations Plans (COOP) and have reviewed them with staff. This expectation to check in does not mean that the health plans need to request copies of providersâ€™ COOP documents."
"After careful consideration, AHCCCS has directed HSAG to suspend all medical record procurement and abstraction activities for the CYE 2018 Hybrid Performance Measure Calculations. At this time, AHCCCS is prioritizing MCO and provider response to COVID-19. Once the COVID-19 emergency response has passed, AHCCCS will re-evaluate project timelines and determine next steps for hybrid audits.",
"No, there are no changes to behavioral health response timeframes for children in DCS custody. For example, the Rapid Response is still required within 72 hours after referral from DCS. Please refer to ACOM 417 and ACOM 449 for additional information on AHCCCS timeframe requirements for children in DCS custody.",
]


In [23]:
mannual_answer=[el.replace('\xa0',' ') for el in mannual_answer]

In [24]:
from itertools import chain

def get_unigram_count(tokens):
    count_dict = dict()
    for t in tokens:
        if t in count_dict:
            count_dict[t] += 1
        else:
            count_dict[t] = 1

    return count_dict


class Rouge:
    beta = 1

    @staticmethod
    def my_lcs_grid(x, y):
        n = len(x)
        m = len(y)

        table = [[0 for i in range(m + 1)] for j in range(n + 1)]

        for j in range(m + 1):
            for i in range(n + 1):
                if i == 0 or j == 0:
                    cell = (0, 'e')
                elif x[i - 1] == y[j - 1]:
                    cell = (table[i - 1][j - 1][0] + 1, '\\')
                else:
                    over = table[i - 1][j][0]
                    left = table[i][j - 1][0]

                    if left < over:
                        cell = (over, '^')
                    else:
                        cell = (left, '<')

                table[i][j] = cell

        return table

    @staticmethod
    def my_lcs(x, y, mask_x):
        table = Rouge.my_lcs_grid(x, y)
        i = len(x)
        j = len(y)

        while i > 0 and j > 0:
            move = table[i][j][1]
            if move == '\\':
                mask_x[i - 1] = 1
                i -= 1
                j -= 1
            elif move == '^':
                i -= 1
            elif move == '<':
                j -= 1

        return mask_x

    @staticmethod
    def rouge_l(cand_sents, ref_sents):
        lcs_scores = 0.0
        cand_unigrams = get_unigram_count(chain(*cand_sents))
        ref_unigrams = get_unigram_count(chain(*ref_sents))
        for cand_sent in cand_sents:
            cand_token_mask = [0 for t in cand_sent]
            cand_len = len(cand_sent)
            for ref_sent in ref_sents:
                # aligns = []
                # Rouge.lcs(ref_sent, cand_sent, aligns)
                Rouge.my_lcs(cand_sent, ref_sent, cand_token_mask)

                # for i in aligns:
                #     ref_token_mask[i] = 1
            # lcs = []
            cur_lcs_score = 0.0
            for i in range(cand_len):
                if cand_token_mask[i]:
                    token = cand_sent[i]
                    if cand_unigrams[token] > 0 and ref_unigrams[token] > 0:
                        cand_unigrams[token] -= 1
                        ref_unigrams[token] -= 1
                        cur_lcs_score += 1

                        # lcs.append(token)

            # print ' '.join(lcs)

            lcs_scores += cur_lcs_score

        # print "lcs_scores: %d" % lcs_scores
        ref_words_count = sum(len(s) for s in ref_sents)
        # print "ref_words_count: %d" % ref_words_count
        cand_words_count = sum(len(s) for s in cand_sents)
        # print "cand_words_count: %d" % cand_words_count

        precision = lcs_scores / cand_words_count
        recall = lcs_scores / ref_words_count
        f_score = (1 + Rouge.beta ** 2) * precision * recall / (recall +
                                                                Rouge.beta ** 2 * precision + 1e-7) + 1e-6  # prevent underflow
        return precision, recall, f_score

In [54]:
import nltk
nltk.download('punkt')

from nltk import word_tokenize
from nltk.translate.bleu_score import sentence_bleu



def get_bleu_score(reference, candidate):
    print(word_tokenize(reference))
    print(word_tokenize(candidate))
    score = sentence_bleu(word_tokenize(reference), word_tokenize(candidate), weights=[0.5, 0.5])
    return '%f' % score

def get_rouge_score(reference_summary, generated_summary):
    r = Rouge()
    [precision, recall, f_score] = r.rouge_l([generated_summary], [reference_summary])
    print("Precision is :" + str(precision) + "\nRecall is :" + str(recall) + "\nF Score is :" + str(f_score))
    return (precision), (recall), (f_score)




[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [55]:
avg_1=0
avg_v1=0
avg_v2=0
avg_v3=0
count1=0
v1=0
v2=0
v3=0
for i in range(0,len(mannual_answer)):
  try:
    system_generated_summary = answer_list1[i][0]
    manual_summmary = mannual_answer[i]
    print((float(get_bleu_score(system_generated_summary, manual_summmary))))
    print(get_rouge_score(system_generated_summary, manual_summmary))
    avg_1=avg_1+(float(get_bleu_score(system_generated_summary, manual_summmary)))
    count1=count1+1
    v1,v2,v3=((get_rouge_score(system_generated_summary, manual_summmary)))
    avg_v1=avg_v1+float(v1)#precision
    avg_v2=avg_v2+float(v2)#recall
    avg_v3=avg_v3+float(v3)#f_score

  except:
    print("Not Compare")

['s']
['Yes', ',', 'the', 'AHCCCS', 'Medical', 'Coding', 'Resources', 'webpage', 'includes', 'a', 'COVID-19', 'Medical', 'Coding', 'Information', 'section', 'and', 'COVID-19', 'Emergency', 'Medical', 'Coding', 'guidance', '.']
0.0
Precision is :0.006622516556291391
Recall is :1.0
F Score is :0.01315889342970927
(0.006622516556291391, 1.0, 0.01315889342970927)
['s']
['Yes', ',', 'the', 'AHCCCS', 'Medical', 'Coding', 'Resources', 'webpage', 'includes', 'a', 'COVID-19', 'Medical', 'Coding', 'Information', 'section', 'and', 'COVID-19', 'Emergency', 'Medical', 'Coding', 'guidance', '.']
Precision is :0.006622516556291391
Recall is :1.0
F Score is :0.01315889342970927
['c']
['Yes', '.', 'The', 'Centers', 'for', 'Medicare', '&', 'Medicaid', 'Services', '(', 'CMS', ')', 'created', 'a', 'new', 'Healthcare', 'Common', 'Procedure', 'Coding', 'System', '(', 'HCPCS', ')', 'code', 'for', 'COVID-19', 'tests', 'administered', 'outside', 'of', 'Centers', 'for', 'Disease', 'Control', 'and', 'Prevention'

In [47]:
avg_1=avg_1/count1

In [48]:
avg_1

0.0047612708333333335

In [49]:
avg_v1=avg_v1/count1
avg_v2=avg_v2/count1
avg_v3=avg_v3/count1


In [51]:
avg_v1

0.0037734259426809563

In [52]:
avg_v2

0.9375

In [53]:
avg_v3

0.007498975504809067